# Insert and tokenize the data

In [1]:
#To run this notebook, you need to have the folder "AI_Course_Final" (downloaded from Kaggle) 
#in the parent folder of the git-repository-folder

In [34]:
%load_ext autoreload
%autoreload 2

In [33]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import copy
import os
from functools import reduce
import pickle
import numpy as np
import h5py

#Imports for the models:
from keras.layers import Dense, Merge, Dropout, Permute, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import RNN, LSTM
from keras.models import Sequential

In [3]:
current = os.getcwd()
print(current) # displays our current directory
parent = os.path.abspath(os.path.join(current, os.pardir)) 
print(parent) # displays the parent directory
path = parent + "/AI_Course_Final/CBTest/data/cbt_test.txt"
print(path) 
os.path.exists(path)
# Check if the wanted path exist. Must be True!!
test = pd.read_csv(path, sep='\n')

/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project/intelligent_computing_project
/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project
/Users/fridahammarberg/Documents/NTHU/intelligent_computing/project/AI_Course_Final/CBTest/data/cbt_test.txt


In [4]:
path_CN_test = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_test_2500ex.txt"
path_CN_train = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_train.txt"
#path_CN_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_CN_valid_2000ex.txt"

path_NE_test = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_test_2500ex.txt"
path_NE_train = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_train.txt"
#path_NE_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_NE_valid_2000ex.txt"

path_P_test = parent + "/AI_Course_Final/CBTest/data/cbtest_P_test_2500ex.txt"
path_P_train = parent + "/AI_Course_Final/CBTest/data/cbtest_P_train.txt"
#path_P_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_P_valid_2000ex.txt"

path_V_test = parent + "/AI_Course_Final/CBTest/data/cbtest_V_test_2500ex.txt"
path_V_train = parent + "/AI_Course_Final/CBTest/data/cbtest_V_train.txt"
#path_V_valid = parent + "/AI_Course_Final/CBTest/data/cbtest_V_valid_2000ex.txt"

In [5]:
pd.set_option('display.max_colwidth', -1)

CN_test = pd.read_csv(path_CN_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
CN_train = pd.read_csv(path_CN_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
#CN_valid = pd.read_csv(path_CN_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

NE_test = pd.read_csv(path_NE_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
NE_train = pd.read_csv(path_NE_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
#NE_valid = pd.read_csv(path_NE_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

P_test = pd.read_csv(path_P_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
P_train = pd.read_csv(path_P_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
#P_valid = pd.read_csv(path_P_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

V_test = pd.read_csv(path_V_test, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
V_train = pd.read_csv(path_V_train, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])
#V_valid = pd.read_csv(path_V_valid, sep='\n', delimiter='\t', header=None, names=['sentence','answer','empty','options'])

In [6]:
test_df = pd.concat([CN_test, NE_test, P_test, V_test])

train_df = pd.concat([CN_train, NE_train, P_train, V_train])

#valid_df = pd.concat([CN_valid, NE_valid, P_valid, V_valid])

In [7]:
test_data = test_df[['sentence']].copy()
train_data = train_df[['sentence']].copy()
#valid_data = valid_df[['sentence']].copy()

In [8]:
# delete the nbrs in the beginning of each sentence
test_data['sentence'] = test_data['sentence'].map(lambda x: x.lstrip('1234567890'))
train_data['sentence'] = train_data['sentence'].map(lambda x: x.lstrip('1234567890'))
#valid_data['sentence'] = valid_data['sentence'].map(lambda x: x.lstrip('1234567890'))

In [9]:
test_answer = test_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)
train_answer = train_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)
#valid_answer = valid_df[['answer']].copy().dropna(axis=0, how='any').reset_index(drop=True)

In [10]:
test_option = test_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)
train_option = train_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)
#valid_option = valid_df[['options']].copy().dropna(axis=0, how='any').reset_index(drop=True)

In [11]:
test_option_list = []
k = 0
for x in test_option.iterrows():
    string_opt = test_option.values[k][0].replace("|", " ")
    test_option_list.append(string_opt)
    k += 1
    
test_option_df = pd.DataFrame(test_option_list, columns=['options'])


train_option_list = []
k = 0
for x in train_option.iterrows():
    string_opt = train_option.values[k][0].replace("|", " ")
    train_option_list.append(string_opt)
    k += 1
    
train_option_df = pd.DataFrame(train_option_list, columns=['options'])

In [12]:
# Do the same to the validation data -- we dont need this during the competition data

#valid_option_list = []
#k = 0
#for x in valid_option.iterrows():
#    string_opt = valid_option.values[k][0].replace("|", " ")
#    valid_option_list.append(string_opt)
#    k += 1
    
#valid_option_df = pd.DataFrame(valid_option_list, columns=['options'])

In [13]:
test_data_list = test_data['sentence'].tolist()
train_data_list = train_data['sentence'].tolist()
#valid_data_list = valid_data['sentence'].tolist()

test_answer_list = test_answer['answer'].tolist()
train_answer_list = train_answer['answer'].tolist()
#valid_answer_list = valid_answer['answer'].tolist()

## Creating a smaller dataset

In [24]:
n = 10
new_test_data_list = copy.deepcopy(test_data_list[:21*2*n])
new_test_answer_list = copy.deepcopy(test_answer_list[:2*n])
new_test_option_list = copy.deepcopy(test_option_list[:2*n])

new_train_data_list = copy.deepcopy(train_data_list[:21*8*n])
new_train_answer_list = copy.deepcopy(train_answer_list[:8*n])
new_train_option_list = copy.deepcopy(train_option_list[:8*n])

## Below we just creates 1 vector for each 21 sentences:

In [25]:
#new_test_data_list/test_data_list to small_test_data_list

t = 0
small_list = []
small_test_data_list = []
for u in new_test_data_list: #change here if we have the big dataset
    if t < 21:
        small_list.append(u)
        t += 1
    else:
        small_test_data_list.append(reduce(lambda a, b: a + b, small_list))
        small_list = []
        t = 1

small_test_data_list.append(reduce(lambda a, b: a + b, small_list))

#***********************************

#new_train_data_list/train_data_list to small_train_data_list

t = 0
small_list = []
small_train_data_list = []
for u in new_train_data_list: #change here if we have the big dataset
    if t < 21:
        small_list.append(u)
        t += 1
    else:
        small_train_data_list.append(reduce(lambda a, b: a + b, small_list))
        small_list = []
        t = 1
small_train_data_list.append(reduce(lambda a, b: a + b, small_list))
#***********************************

# Tokenizer

In [26]:
tokenizer = Tokenizer()

In [27]:
#tokenizer.fit_on_texts(train_data_list)
#tokenizer.fit_on_texts(new_train_data_list)
tokenizer.fit_on_texts(small_train_data_list)

In [28]:
train_data_matrix = tokenizer.texts_to_matrix(small_train_data_list)
train_answer_matrix = tokenizer.texts_to_matrix(new_train_answer_list)
train_options_matrix = tokenizer.texts_to_matrix(new_train_option_list)

In [29]:
test_data_matrix = tokenizer.texts_to_matrix(small_test_data_list)
test_answer_matrix = tokenizer.texts_to_matrix(new_test_answer_list)
test_options_matrix = tokenizer.texts_to_matrix(new_test_option_list)

## Increse the dimension:

In [30]:
best_train_answer_matrix = np.expand_dims(train_answer_matrix, -1)
best_test_answer_matrix = np.expand_dims(test_answer_matrix, -1)

# Creating the first model - RNN

In [35]:
#Constants
VOCABULARY_SIZE = 729 
HIDDEN_LAYER_SIZE = 10  
BATCH_SIZE = 20 
EPOCHS_SIZE = 5

memory_encoder = Sequential()
memory_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE, input_length = 729))
memory_encoder.add(Dropout(0.3))

answer_encoder = Sequential()
answer_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE, input_length = 729))
answer_encoder.add(Dropout(0.3))

#Dot product between answer and question encoders, produces internal state "u"
prediction = Sequential()
prediction.add(Merge([memory_encoder, answer_encoder], mode ='dot'))
prediction.add(Dropout(0.3))
prediction.add(Dense(10)) #for regularization
prediction.add(Activation('softmax')) #for softmax 

#Optimizer, loss function, metrics setting
prediction.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training
prediction.fit([train_data_matrix, train_options_matrix], best_train_answer_matrix, batch_size=BATCH_SIZE, 
           nb_epoch=EPOCHS_SIZE, validation_data=([test_data_matrix, test_options_matrix], best_test_answer_matrix))

#scores = prediction.evaluate([test_data_matrix, test_options_matrix], best_test_answer_matrix)
#print("%s: %.2f%%" % (prediction.metrics_names[1], scores[1]*100))

/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 5s 61ms/step - loss: 2.2973 - acc: 0.5374 - val_loss: 2.2855 - val_acc: 0.9997
Epoch 2/5
80/80 [==============================] - 1s 12ms/step - loss: 2.2765 - acc: 0.9605 - val_loss: 2.2565 - val_acc: 0.9997
Epoch 3/5
80/80 [==============================] - 1s 12ms/step - loss: 2.2440 - acc: 0.9981 - val_loss: 2.2135 - val_acc: 0.9997
Epoch 4/5
80/80 [==============================] - 1s 12ms/step - loss: 2.1971 - acc: 0.9986 - val_loss: 2.1522 - val_acc: 0.9997
Epoch 5/5
80/80 [==============================] - 1s 12ms/step - loss: 2.1312 - acc: 0.9986 - val_loss: 2.0665 - val_acc: 0.9997


# Creating the second model - end to end memory network

In [36]:
#Constants
VOCABULARY_SIZE = 729 #53628 - estimate from the goldilocks paper
HIDDEN_LAYER_SIZE = 64  # 10 neurons for 10 options
LSTM_OUTPUT_SIZE=10 #TODO: change LSTM SIZE
BATCH_SIZE = 32
EPOCHS_SIZE = 5

#Initial Embedding matrices A, B for story and options
#Each sentence is taken as an n-dimensional vector of float64, with n=VOCABULARY-SIZE
#memory_encoder encodes lines 1-21 from story, answer encoder
memory_encoder = Sequential()
memory_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length = 729)) #input length = 21
memory_encoder.add(Dropout(0.3))

option_encoder = Sequential()
option_encoder.add(Embedding(input_dim = VOCABULARY_SIZE, output_dim = HIDDEN_LAYER_SIZE,input_length = 729)) #input length = 1
option_encoder.add(Dropout(0.3))

#Dot product between answer and question encoders, produces internal state "u"
match = Sequential()
match.add(Merge([memory_encoder, option_encoder], mode="dot", dot_axes=[2, 2]))

#Story embedded with matrix C to produce output vectors
output_memory_encoder = Sequential()
output_memory_encoder.add(Embedding(input_dim=VOCABULARY_SIZE,output_dim = 729,input_length = 729)) #input length = 21
output_memory_encoder.add(Dropout(0.3))

# Response vector, obtained by summing output vectors
# Output dim: (None, 10 options, 21)
response_vector = Sequential()
response_vector.add(Merge([match, output_memory_encoder], mode="sum"))
response_vector.add(Permute((2, 1)))

#Final embedding for prediction, weighting matrix W
#Combine response and question vectors and do backpropagation
prediction = Sequential()
prediction.add(Merge([response_vector, option_encoder], mode="concat", concat_axis=-1))
#prediction.add(LSTM(LSTM_OUTPUT_SIZE)) #for soft memory attention mechanism
prediction.add(Dropout(0.3))
prediction.add(Dense(10)) #for regularization
prediction.add(Activation("softmax")) #for softmax 

prediction.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

prediction.fit([train_data_matrix, train_options_matrix, train_data_matrix], best_train_answer_matrix, batch_size=BATCH_SIZE, 
           nb_epoch=EPOCHS_SIZE, validation_data=([test_data_matrix, test_options_matrix, test_data_matrix], best_test_answer_matrix))


/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fridahammarberg/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 5s 58ms/step - loss: 2.2863 - acc: 0.2694 - val_loss: 2.2492 - val_acc: 0.5926
Epoch 2/5
80/80 [==============================] - 3s 41ms/step - loss: 2.2156 - acc: 0.5908 - val_loss: 2.1311 - val_acc: 0.7219
Epoch 3/5
80/80 [==============================] - 3s 41ms/step - loss: 2.0774 - acc: 0.7843 - val_loss: 1.9314 - val_acc: 0.8606
Epoch 4/5
80/80 [==============================] - 3s 40ms/step - loss: 1.8565 - acc: 0.9208 - val_loss: 1.6456 - val_acc: 0.9889
Epoch 5/5
80/80 [==============================] - 3s 40ms/step - loss: 1.5536 - acc: 0.9889 - val_loss: 1.2831 - val_acc: 0.9997
